<a href="https://colab.research.google.com/github/karenswang/computational-journalism/blob/main/Computational_Journalism_Meeting_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://pbs.twimg.com/profile_images/1685001219469647872/5T9vglL1_400x400.png" width=150>

## Community Notes: Ice Cream in Kyiv

This notebook will cover the basics of (relational) database design, some pandas gymnastics and a little on content moderation... or one automated approach to it.

[Community Notes (born Birdwatch) is a  program offered by X](https://twitter.com/i/birdwatch) that invites "community" members to provide useful context for tweets. It is an attempt to deal with misinformation on the platform.

Here's a screenshot of a tweet with a note. The tweet is criticizing President Biden for answering serious questions from reporters while eating an ice cream cone. The tweet has numerical ID  `1762285247922614315` -- all tweets are uniquely identifiable by their IDs.

<br>

<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-27%20at%2012.49.51%20PM.png?raw=true" width=400>
<br><br>

The tweet is from a commentator Clay Travis ([X profile](https://twitter.com/ClayTravis), [website](https://www.outkick.com/). On X, Travis has a numerical ID of `50772918`. Every person on X has an ID. For a long time, IDs were assigned sequentially... literally. So the smaller your number, the higher your status in some sense.

For context on the Bident comment from Travis, here the tweet pinned his timeline.





In [1]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Trump smokes Haley, Biden ominous signs in Michigan primary, Bud Light keeps tanking, Super Bowl ads don’t work, Athens mayor says Trump’s to blame for Laken Riley murder, Mitch McConnell steps down, NFL combine under way, CBB picks with prize picks<a href="https://t.co/SxcKEBVgRA">https://t.co/SxcKEBVgRA</a></p>&mdash; Clay Travis (@ClayTravis) <a href="https://twitter.com/ClayTravis/status/1762931219560362107?ref_src=twsrc%5Etfw">February 28, 2024</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Under the tweet about Biden, you see a "note" authored by a Community Notes participant, a "contributor." The note is meant to clarify the tweet, in this case pointing out that the president was taken for an ice cream after his appearance on Late Night with Seth Meyers.

Anyone can apply for the Community Notes program, and once enrolled, you can write a note for any tweet you see. Again, for (flimsy) context, since the program started in 2022, there have been 45 notes about ice cream, and 11 of those involved Biden. Here's another.



In [2]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Dark Brandon: “If I want to eat an ice cream cone in Kyiv, I’ll eat an ice cream cone in Kyiv.” <a href="https://t.co/CeAfdMPWWC">pic.twitter.com/CeAfdMPWWC</a></p>&mdash; Dead End (@RegimeChangeInc) <a href="https://twitter.com/RegimeChangeInc/status/1627699857463947264?ref_src=twsrc%5Etfw">February 20, 2023</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In this case, the tweet has an ID of `1627699857463947264`, the author RegimeChangeInc has an ID of `867194634736472067` and the note itself has an id `1628599268301840386`. (Yes, I'm emphasizing the IDs.)

Anyone can become a contributor to the program. Well, you have to have been active for 6+ months, you can't have any recent rule violations and you have to have a verified phone number. It all that checks out,   sign up [here](http://twitter.com/i/flow/join-birdwatch)

<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-29%20at%2011.19.48%20AM.png?raw=true" width=300>
<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-29%20at%2011.19.56%20AM.png?raw=true" width=300>
<br>

<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-29%20at%2011.20.10%20AM.png?raw=true" width=300>
<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-29%20at%2011.20.18%20AM.png?raw=true" width=300>

And on the [X user guide describing the program](https://communitynotes.x.com/guide/en/about/introduction), they provide more context.


<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-29%20at%2012.32.40%20PM.png?raw=true" width=500>

**Focus on the code**

So code and data are available! Amazing! Let's start with the code. There were a lot of terms in the description of the program that need some kind of technical definition. Maybe the most important being that judgements about the visibility of notes depends on the actions of contributors with "diverse perspectives."

Here's a snippet from [a great explanation](https://vitalik.eth.limo/general/2023/08/16/communitynotes.html) of how the notes process works.

>Once you are accepted, you can at first participate in rating existing notes as being "helpful" or not, and once you've made enough good ratings ([measured](https://communitynotes.twitter.com/guide/en/under-the-hood/contributor-scores) by seeing which ratings match with the final outcome for that note), you can also write notes of your own.
<br><br>
When you write a note, the note gets a score based on the reviews that it receives from other Community Notes members. These reviews can be thought of as being votes along a 3-point scale of `HELPFUL`, `SOMEWHAT_HELPFUL` and `NOT_HELPFUL`... Based on these reviews, a note gets a score. If the note's score is above [a threshold], the note is shown; otherwise, the note is not shown.
<br><br>
The way that the score is calculated is what makes the algorithm unique. Unlike simpler algorithms, which aim to simply calculate some kind of sum or average over users' ratings and use that as the final result, **the Community Notes rating algorithm explicitly attempts to prioritize notes that receive positive ratings from people across a diverse range of perspectives.** That is, if people who usually disagree on how they rate notes end up agreeing on a particular note, that note is scored especially highly.

This is how the tweet by RegimeChangeInc about Biden in Kyiv received its note. Many tweets that have publicly visible notes end up disappearing. Funny that. You can see latest notes activity on this [X page](http://twitter.com/i/birdwatch).

<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-27%20at%2012.51.44%20PM.png?raw=true" width=500>

>Let us get into the deep math of how this works. We have a set of users and a set of notes; we can create a matrix M, where the cell Mij represents how the i'th user rated the j'th note.
<br>
<img src="https://vitalik.eth.limo/images/communitynotes/matrix.png" width=400>
<br>
For any given note, most users have not rated that note, so most entries in the matrix will be zero, but that's fine. The goal of the algorithm is to create a four-column model of users and notes, assigning each user two stats that we can call "friendliness" and "polarity", and each note two stats that we can call "helpfulness" and "polarity". The model is trying to predict the matrix as a function of these values, using the following formula:
<br>
<img src="https://vitalik.eth.limo/images/communitynotes/formula.png" width=400>
<br>
Note that here I am introducing both the terminology used in the Birdwatch paper, and my own terms to provide a less mathematical intuition for what the variables mean:
<br>
* μ is a **"general public mood"** parameter that accounts for how high the ratings are that users give in general
* iu is a user's **"friendliness"**: how likely that particular user is to give high ratings
* in is a note's **"helpfulness"**: how likely that particular note is to get rated highly. **Ultimately, this is the variable we care about.**
* fu or fn  is user or note's **"polarity"**: its position among the dominant axis of political polarization. In practice, negative polarity roughly means "left-leaning" and positive polarity means "right-leaning", but note that **the axis of polarization is discovered emergently from analyzing users and notes; the concepts of leftism and rightism are in no way hard-coded.**

>**The core clever idea here is that the "polarity" terms absorb the properties of a note that cause it to be liked by some users and not others, and the "helpfulness" term only measures the properties that a note has that cause it to be liked by all.** Thus, selecting for helpfulness identifies notes that get cross-tribal approval, and selects against notes that get cheering from one tribe at the expense of disgust from the other tribe.










**To the data!**

Now, let's get back to the data. On the [Notes data site](https://twitter.com/i/birdwatch/download-data) you can find links to four kinds of data sets.


+ **Notes** - These are context people provide for tweets
+ **Ratings of Notes** - These score notes as being helpful or not
+ **History of each Note** - After notes accumulate enough ratings they can be shown on X, providing a word of caution or a vote of confidence. This table tracks the progress of notes over time.
+ **User Enrollment**  - A table with metadata about each user's enrollment state

There is also a [guide to analyzing Notes data](https://twitter.com/i/birdwatch/download-data) that has a pointer to the data dictionary. To grab data for this drill, I went to [this page](https://twitter.com/i/communitynotes/download-data) and "right clicked" on the Notes data set. I copied the link to the data and pasted it in the `read_csv()` command below. Oh and the data are tab-separated so I added the argument `sep="\t"` to the `read_csv()` call.

In [3]:
import pandas as pd

# use set_option() to remove the limits from the number of columns
# of a dataframe that can be shown and the number of characters in each cell
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# read in the notes and see what the columns are!
notes = pd.read_csv("https://ton.twimg.com/birdwatch-public-data/2024/02/27/notes/notes-00000.tsv",sep="\t")
notes.columns

<ipython-input-3-d4bd67b94081>:9: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv("https://ton.twimg.com/birdwatch-public-data/2024/02/27/notes/notes-00000.tsv",sep="\t")


Index(['noteId', 'noteAuthorParticipantId', 'createdAtMillis', 'tweetId',
       'classification', 'believable', 'harmful', 'validationDifficulty',
       'misleadingOther', 'misleadingFactualError',
       'misleadingManipulatedMedia', 'misleadingOutdatedInformation',
       'misleadingMissingImportantContext', 'misleadingUnverifiedClaimAsFact',
       'misleadingSatire', 'notMisleadingOther',
       'notMisleadingFactuallyCorrect',
       'notMisleadingOutdatedButNotWhenWritten', 'notMisleadingClearlySatire',
       'notMisleadingPersonalOpinion', 'trustworthySources', 'summary',
       'isMediaNote'],
      dtype='object')

In [4]:
notes.shape

(610377, 23)

That's a lot of notes!

Let's look at the data. We are going to just focus on a couple columns, however, to keep things readable.

In [5]:
# have a look at some of the notes. i'm keeping just a few of the columns
# to make things easier to read.

notes[["classification", "summary","noteAuthorParticipantId","tweetId","createdAtMillis"]]

,classification,summary,noteAuthorParticipantId,tweetId,createdAtMillis
0,MISINFORMED_OR_POTENTIALLY_MISLEADING,Forbes has a good rundown of the investigation and the Washington Post has a fuller picture of where the investigation is headed. Gaetz seems to be deliberately misleading his readers about the timeline of any investigation with this tweet. https://www.forbes.com/sites/rachelsandler/2021/03/30/gop-rep-matt-gaetz-reportedly-under-investigation-over-relationship-with-17-year-old-girl/?sh=7da3be1a23f4 https://www.washingtonpost.com/politics/2022/01/27/sex-trafficking-allegations-matt-gaetz/,5684B38EB58FD8BE75ABA37F0BE040EC70380B002ADF9DBDE2415CB32782EAEE,1377030478167937024,1655318404027
1,NOT_MISLEADING,They are expressing a personal opinion in a straightforward manner. This should not need a note.,5684B38EB58FD8BE75ABA37F0BE040EC70380B002ADF9DBDE2415CB32782EAEE,1536848327979016193,1655318986910
2,MISINFORMED_OR_POTENTIALLY_MISLEADING,Teslas purchased after 12/31/19 are not eligible for US Federal tax credits because they exceeded the initial 200K eligible cars allowed a credit. States like CA have their own programs that issue rebates or credits but many have ended as well due to higher Tesla prices. https://cleanvehiclerebate.org/en/faqs/can-i-apply-rebate-my-tesla-model-3-or-tesla-model-y,5684B38EB58FD8BE75ABA37F0BE040EC70380B002ADF9DBDE2415CB32782EAEE,1537080831751102467,1655319460217
3,MISINFORMED_OR_POTENTIALLY_MISLEADING,The Jan 6th riots were encouraged by the sitting US President saying to his followers to &quot;stop the steal&quot; not just protest it. As well as laying groundwork well before. https://www.nytimes.com/2021/01/10/us/trump-speech-riot.html https://www.wsj.com/articles/trump-and-his-allies-set-the-stage-for-riot-well-before-january-6-11610156283 Four people in the crowd on Jan 6th died as well as five officers shortly after. https://www.nytimes.com/2022/01/05/us/politics/jan-6-capitol-deaths.html,5684B38EB58FD8BE75ABA37F0BE040EC70380B002ADF9DBDE2415CB32782EAEE,1537196168953974784,1655333070821
4,MISINFORMED_OR_POTENTIALLY_MISLEADING,"The Committee has been found by numerous courts to be constitutional &amp; is not losing the interest of the American people. CNN reports &lt; 20M viewers tuned in to Thursday's presentation about the Jan 6 attack, and it reached a far larger number through social and others. https://www.cnn.com/2022/06/10/media/ratings-january-6-hearings/index.html",5684B38EB58FD8BE75ABA37F0BE040EC70380B002ADF9DBDE2415CB32782EAEE,1540087463099736065,1656100269455
...,...,...,...,...,...
610372,MISINFORMED_OR_POTENTIALLY_MISLEADING,Non è possibile donare spunte blu a meno che non si tratti di organizzazioni verificate (ma non è questo il caso). È un imbroglio per ottenere più interazioni. business.twitter.com,AD436976FE2B66492611D4E08E5CCA144AFADF4FFD055ECD600990C0C8C8D2B9,1759624449886650695,1708522396933
610373,MISINFORMED_OR_POTENTIALLY_MISLEADING,"Non è possibile donare spunte blu, a meno che non si tratti di un'organizzazione verifica, ma non è questo il caso. Questa truffa sta girando per far sì che si ottengano più interazioni. verified twitter.com",AD436976FE2B66492611D4E08E5CCA144AFADF4FFD055ECD600990C0C8C8D2B9,1759647844204433719,1708757238434
610374,MISINFORMED_OR_POTENTIALLY_MISLEADING,PTI is the largest political party in the country that represents the people of Pakistan &amp; has made every effort to ensure people's stolen mandate is restored. Such tweets are spread to make people feel helpless &amp; hopeless &amp; shouldn't be tolerated in this platform. https://www.reuters.com/world/asia-pacific/pakistans-imran-khan-backed-candidates-join-sunni-party-interim-party-chief-2024-02-19/ https://x.com/PTIofficial/status/1758145839392760000?s=20 https://foreignpolicy.com/2024/02/14/pakistan-elections-pti-imran-khan-upset-military/ https://x.com/WorldTimesWT/status/1756705625260609895?s=20,7DEFE4FA6920E805F21609ACE16DD20D6531E932D6728B7DC78F89D3445F625C,1759847303576449044,1

The [data dictionary](https://communitynotes.x.com/guide/en/under-the-hood/download-data) explains that the `classification` of a note is selected by it's author to indicate whether the tweet they are commenting on is "NOT MISLEADING" or "MISINFORMED OR POTENTIALLY MISLEADING". The texts of the notes is given in the column `summary` and its author is specified by the `participantId`. This is not a person's X ID, but another unique identifier that is unique to them within the Notes universe. The `tweetId` on the other hand specifies the unique tweet ID we looked at above. You can see a tweet by  adding the ID to this URL...


```
http://twitter.com/anyuser/status/
```

For example, to see the tweet referenced in the first row of the dataframe, the tweet with `tweetID` `1377030478167937024`, you can enter the URL

```
http://twitter.com/anyuser/status/1377030478167937024
```

into your browser. Finally, the `createdAtMillis` column gives you date and time the note was created. As mentioned in class, this represents the number of milliseconds since midnight on January 1, 1970, the so-called Unix Epoch. (We've seen that before.)

Now, back to the data.

**1. Each note is written by one Community Notes participant. But a participant might write many notes. How many notes did the most prolific participant create?**

**2. Each note is about only one tweet, but there can be many notes about the same tweet. Which tweet had the most notes? What was it about?**

In [ ]:
# some exploratory here



This work is going to get interesting if we can bring in the  text of each note as well as it's `summary`. It is irresistable data. What kinds of words tend to be used in flagged tweets? What kinds of comments are seen as helpful?

Starting slowly, in our first class we saw that string objects had certain methods they like `.upper()`. We can invoke methods like these on all the strings in a column of a DataFrame by using `.str` after the column name.

Here, we turn the summaries uppercase. We call the new Series `caps`.

In [ ]:
caps = notes["summary"].str.upper()
caps

If we wanted to count the occurrences of "LGBTQ" we could create a `True`/`False` column, `True` if the summary contains "LGBTQ" and `False` otherwise.  The Series `caps` is again made up of strings so we can use the string method `.contains()` to test for COVID. We'll call the new  Series `lgbtq`.

In [ ]:
lgbtq = caps.str.contains("LGBTQ")
lgbtq

We can then create a count using `.value_counts()` ...

In [ ]:
lgbtq.value_counts()

... or by even just adding up the logical data, with `True` becoming 1 and `False` becoming 0.

In [ ]:
lgbtq.sum()

If we wanted to, we could do this all in one line... This is either super exciting or really off-putting. It's about readability. There are lots of ways to ask Python for a computation, and you'd like to make your code understandable by others.

Fun fact -- the person most likely to be confused by your code is you, six months from now.

In [ ]:
notes["summary"].str.upper().str.contains("LGBTQ").sum()

**3. Fill out the COVID topic with other keywords and look for their presence in the summaries. Or try another topic altogether.**

In [ ]:
# your code here



A comment on dates. As we mentioned in class, the `createdAtMillis` represents the time a note was created. But it measures time in terms of time since an epoch. This is a different way of thinking about time. We have seens  days of the form M/D/Y...

<pre>
02/27/2024
02/10/2024
01/28/2024
12/31/2023
</pre>

... and Y-M-D.

<pre>
2024-02-01
2024-02-01
2024-02-15
2024-02-15
2024-02-22
</pre>

Is there any difference? What is easy to do with these two formats and what is hard? Recall our discussion in class.

Now instead of these two formats, the `notes` DataFrame has dates that look like large numbers. They record seconds since the Unix Epoch, or midnight January 1, 1970. Yes, that's arbitrary, but any epoch will do -- everything after is represented as positive milliseconds since the epoch, and everything before takes on negative values.

<pre>
1655318404027
1655318986910
1655319460217
1655333070821
1656100269455
</pre>

Or with a little effort, here's what these numbers translate to in terms of dates and times.

```
Epoch date      Human-readable date (GMT)
1655318404027   2022-06-15 18:40:04.027
1655318986910   2022-06-15 18:49:46.910
1655319460217   2022-06-15 18:57:40.217
1655333070821   2022-06-15 22:44:30.821
1656100269455   2022-06-24 19:51:09.455
```

Again, recall our discussion in class. What does the numeric representation on the left provide that the symbolic representation on the right does not?
<br>
<br>

<img src="https://github.com/cocteau/Computational_Journalism_2024/blob/main/images/Screenshot%202024-02-27%20at%203.01.59%20PM.png?raw=true" width=200>
<br>
<br>

We can convert the milliseconds `createdAtMillis` to a readable "datetime" object using a function inside pandas called (predictably enough) `to_datetime()`. Here's how we might use it.

The code below creates a new column called `createdAtDatetime`. You can see it by scrolling the display below all the way to the right.

In [ ]:
import pandas as pd
# here we convert the millisecond column, making sure
# to tell the program our big numbers are in milliseconds
# (and not just plain seconds, say)

# add a new column called `datetime` and look for it at the right of the display.
# We use the argument unit="ms" to specify our data are milliseconds since
# the Unix Epoch
notes["createdAtDatetime"] = pd.to_datetime(notes["createdAtMillis"],unit="ms")
notes["createdAtDatetime"]

In [ ]:
# have another look at some of the notes. again, i'm keeping just a few of the columns
# to make things easier to read.

notes[["classification", "summary","noteAuthorParticipantId","tweetId","createdAtDatetime"]]

The new column holds data of type `datetime`. A datetime object has lots of methods and data associated with it. In a DataFrame, you can get access to these  by explicitly inserting a `.dt` after your `createdAtDatetime` column. As we have seen, the same thing holds if you have a column of `str` or string objects, and we used `.str` in front of any of the string methods you learned like `.upper()`.

Here we pull out parts of the date and time a note was created.

In [ ]:
# create a column of the months for each note
notes["createdAtDatetime"].dt.month

In [ ]:
# create a column for the hours for each note
notes["createdAtDatetime"].dt.hour

The `.date` attribute of a `datetime` object holds the calendar date (Y-M-D), dropping the time component.

In [ ]:
notes["createdAtDatetime"].dt.date

Let's use this last expression to create a column called `"date"` in `notes` to hold just the day each note was created. This way we can make simple counts of notes per day to judge how active the Community guardians are.

In [ ]:
# if we just want the day the note was posted and not the time, we
# can access the datetime methods using .dt -- here we pull the calendar date
# the note was created. We use the "unit" of "ms" to indicate our big number
# is counting milliseconds since the epoch
notes["date"] = notes["createdAtDatetime"].dt.date
notes.head()


Let's have a closer look again.

In [ ]:
notes[["summary","classification","noteAuthorParticipantId","tweetId","date"]]

Now let's count notes per day. Is there an increase in the use of (what was) Birdwatch since Musk took over the company? Let's introduce a new friend `groupby()` to group the notes data by day and then count how many we have per day. Then we'll make a barplot, one bar per day, the heights recording the number of notes introduced on that day.

In [ ]:
perday = notes.groupby("date",as_index=False)["noteId"].count()
perday

In [ ]:
from plotly.express import bar

bar(perday,x="date",y="noteId")

You can see an overall rise in time. That's a lot of activity.

Let's focus on notes from just the last month. We can do this by comparing our datetime column `createdAtDatetime` against a string representing a date (and possibly time). Here we take all the notes created after Feb 1 (the current data runs through Sunday the 25th I believe).

(This special comparison with a string like `"2024-02-01"` works with a datetime object and not just the straight `date` column we created.)

In [ ]:
newnotes = notes[notes["createdAtDatetime"] > "2024-02-01"]
newnotes.shape

Let's have a look at the tweets that had the most notes created this week!

In [ ]:
newnotes.value_counts("tweetId")

The top two are both from Tucker Carlson about his interview of Vladimir Putin. The third was about a post from Musk, the  owner of X. Here's the three tweets.

In [ ]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Ep. 73 The Vladimir Putin Interview <a href="https://t.co/67YuZRkfLL">pic.twitter.com/67YuZRkfLL</a></p>&mdash; Tucker Carlson (@TuckerCarlson) <a href="https://twitter.com/TuckerCarlson/status/1755734526678925682?ref_src=twsrc%5Etfw">February 8, 2024</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [ ]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Why I&#39;m interviewing Vladimir Putin. <a href="https://t.co/hqvXUZqvHX">pic.twitter.com/hqvXUZqvHX</a></p>&mdash; Tucker Carlson (@TuckerCarlson) <a href="https://twitter.com/TuckerCarlson/status/1754939251257475555?ref_src=twsrc%5Etfw">February 6, 2024</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [ ]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">This is a test</p>&mdash; Elon Musk (@elonmusk) <a href="https://twitter.com/elonmusk/status/1760446632678834540?ref_src=twsrc%5Etfw">February 21, 2024</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

**4. Write some code to see the notes for each one of these  tweets. What kind of comment was made for each?**

In [ ]:
# Your code here



**Introducing group-by**

First, read in the data again, just to make sure we've got everything.

In [ ]:
import pandas as pd

# use set_option() to remove the limits from the number of columns
# of a dataframe that can be shown and the number of characters in each cell
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# read in the notes and see what the columns are!
notes = pd.read_csv("https://ton.twimg.com/birdwatch-public-data/2024/02/27/notes/notes-00000.tsv",sep="\t")

# turning miliseconds into a datetime object
notes["createdAtDatetime"] = pd.to_datetime(notes["createdAtMillis"],unit="ms")

# and just look at the date component
notes["date"] = notes["createdAtDatetime"].dt.date
notes.head()





As we have seen so far, a lot of our analytical work has involved *summarization*. That is, computing summary statistics like `sum()`, `mean()`, `median()`, `min()`,  `max()` and even `count()` over aggregations of our data set. We have filtered the data in various ways

* Studying the notes on a tweet
* Comparing effectiveness of different note takers

Here are some simple summary statistics we have computed...

In [ ]:
# number of notes referencing LGBTQ issues
notes["summary"].str.upper().str.contains("LGBTQ").sum()

In [ ]:
# and the fraction
notes["summary"].str.upper().str.contains("LGBTQ").mean()

In [ ]:
# the fraction declared misleading because the tweets are satire
notes["misleadingSatire"].mean()

Similarly, for categorical data, we've used `value_counts()` to tabulate the number of items of each category.

In [ ]:
notes.value_counts("tweetId")

The last calculation is dividing the `notes` dataframe by tweets, to separate our data by `tweetId`. Then, to each group we `count()` the number of notes associated with each `tweetID`.

The **group-by** operation extends these ideas, and lets us aggregate our data based on some label or the values of one or more columns in our data. In words, the group-by operation *splits* the data according to some condition, *applies* some aggregation within those groups, and then *combines* the data back together.

In pictures, this is how group-by works.

![Split Apply Combine figure](https://github.com/altair-viz/altair-tutorial/blob/master/notebooks/split-apply-combine.png?raw=1)
[Figure source](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)

So let's look at how pandas implements this group-by operation using the method `groupby()`. First, let's split the data into groups based on `AnimalGroupParent`. We'll just use the `lfb` DataFrame, and call `groupby()` with this column name.



In [ ]:
tweet_groups = notes.groupby("tweetId")
tweet_groups

As we have seen, this returns a `DataFrameGroupBy` object. Stepping ahead just a little, we can examine what this structure looks like. Literally it has created groups by dividing the `note` DataFrame into pieces. One piece per tweet in the `tweetId` column.

To see this we will *iterate* or *loop* over the groups using a `for` loop. Let's run it and talk about what it is doing after.

In [ ]:
# loop over the groups formed by groupby(). Each has a
# label (a tweetId) and then a DataFrame holding all
# the rows relating to notes about this tweet

for tweet,df in tweet_groups:
  print("---"*10)
  print("First five rows of group for tweets -", tweet)
  print(df.head(5))

Each tweetId is associated with a DataFrame that has just the rows from `notes` that involve the given tweet.

The display above starts with a separator "-----------". It then has a message that what follows are the first five rows of the DataFrame associated with a particular tweet. It then prints out the first five rows (assuming there are five). The printout of the rows is long and not what we're used to seeing. But scroll through the output and start with a "-----------" and read it over. Make sure you understand that we're looking at pieces of our `notes` DataFrame broken down by tweetId.

We can grab one of these group's DataFrame using `get_group()`. Here's one involving a Tesla, with `tweetId`  `1719444510416032022`.

In [ ]:
tweet_groups.get_group(1719444510416032022)

To be clear, this is all the notes associated with `tweetId` `1719444510416032022` from `notes`. That should be the same as if we manually extracted just the the notes from `notes` using the subsetting we learned.

In [ ]:
notes[notes["tweetId"]==1719444510416032022]

From the grouped data structure, we can do all the subsetting we might usually do in pandas. Here we pull the column `createdAtMillis` from the group of notes.

In [ ]:
tweet_groups.get_group(1719444510416032022)["createdAtMillis"]

Which compares to what we would do if we manually extracted the column of times from the DataFrame of just notes about tweet `1719444510416032022`.

In [ ]:
notes[notes["tweetId"]==1719444510416032022]["createdAtMillis"]

Finally, to each group we can apply one of our summary statistics like `sum()` and `std()` and `count()`.

Here is one summary, the number of rows or the *count* of items in our aggregate, this time just applied first to the group corresponding to `tweetId` `1719444510416032022`...

In [ ]:
tweet_groups.get_group(1719444510416032022)["createdAtMillis"].count()

... and then removing `.get_group()` and applying it to all the groups.

In [ ]:
tweet_groups["createdAtMillis"].count()

(Which should look like the output from `.value_counts()`.) We can also select the column `notMisleadingFactuallyCorrect` and look at the fraction of notes about the tweet that were thought to be factually correct.

In [ ]:
tweet_groups["notMisleadingFactuallyCorrect"].mean()

We can do these calls in one line. That is, instead of forming `tweet_groups`, we can do all this in one command.

In [ ]:
notes.groupby("tweetId")["createdAtMillis"].count()


Or maybe instead we summarize the aggregate with `min()`, meaning the first time a note appeared about the tweet.

In [ ]:
notes.groupby("tweetId")["createdAtMillis"].min()

In [ ]:
notes.groupby("tweetId")["notMisleadingFactuallyCorrect"].mean()

This is incredibly powerful for our analysis!

Let's push this a little farther. First, compute the number of notes per participant. We group-by `noteAuthorParticipantId` and then `count()` the number of notes per person.

In [ ]:
notes.groupby("noteAuthorParticipantId")["noteId"].count()

The result here is a Series. The title `noteAuthorParticipantId` is playing the role of an index labeling each entry in the Series. We can keep the `noteAuthorParticipantId` as a column in a DataFrame by using the argument `as_index=False`.

Here's the same call as the one above but with this new argument added.

In [ ]:
notes.groupby("noteAuthorParticipantId",as_index=False)["noteId"].count()

This way, we have a DataFrame with columns `noteAuthorParticipantId` and `noteId` to hold the counts of notes per participant.

The summary statistics  we learned about earlier can be applied to a single column - here we look at the proportion of tweets each participant marked as factually correct.

In [ ]:
notes.groupby("noteAuthorParticipantId",as_index=False)["notMisleadingFactuallyCorrect"].mean()

... or to multiple columns - here we look at the fraction of times each contributor marked a tweet as factually correct or factually incorrect.

In [ ]:
notes.groupby("noteAuthorParticipantId",as_index=False)[["notMisleadingFactuallyCorrect","misleadingFactualError"]].mean()

Finally, we can also group-by the unique combinations of values from two or more columns. This is like what we did with `.value_counts()`.

Here we look at grouping by both participant and year, tallying total notes and total tweets declared factually incorrect. (Compare the code in the next cell to the one above.)

In [ ]:
# use the datetime column to pull out the year
notes["year"] = notes['createdAtDatetime'].dt.year

notes.groupby(["noteAuthorParticipantId","year"],as_index=False)[["notMisleadingFactuallyCorrect","misleadingFactualError"]].mean()

So far, we have applied one summary to all the columns we specify. Instead, we can compute any kind of summary statistic for each column. Here we use the method `.agg()` to be very explicit about what we are wanting to compute. This method takes "tuples" with column names and  summary statistics as the associated values.

The result below is a new DataFrame called `contributors` that has columns of the number of notes each participant created and the time of their first and last ones.

In [ ]:
# use .agg() to specify how to summarize each column
contributors = notes.groupby("noteAuthorParticipantId",as_index=False).agg(count=("noteId","count"),
                                                                           start=("createdAtDatetime","min"),
                                                                           end=("createdAtDatetime","max"))
contributors

**Bringing in the Tweets**

In terms of DataFrames, the character missing from the story so far are the tweets themselves. Right now we're looking them up manually.

We can use [Twitter's Application Programming Interface (API)](https://developer.twitter.com/en/products/twitter-api) to pull the  tweets that had notes written about them this month. We spent a lot of time talking about APIs -- they are an easy way to ask a web site for data. The "I" for Interface refers the well-documented way you can ask for data and the consistent format that is used to deliver what you asked for.

I used the Twitter API to pull 1400 of the top noted tweets (the API has rate limits and I cut it off figuring it would take a while to get all I needed).

Still, let's read it in.

In [ ]:
from pandas import read_csv

tweets = read_csv("https://github.com/cocteau/Computational_Journalism_2024/raw/main/data/tweets.csv")

In [ ]:
tweets.head()

**5. Who is responsible for the greatest number of these top 1400 tweets. Who came in second? Third?**

In [ ]:
# Your code here

**6. Select a tweet (maybe one with a lot of retweets or one from someone you follow) and extract the notes on it from your DataFrame `newnotes`.**

In [ ]:
# Your code



Finally, let's now consider the ratings data. It is accessible from the same page that our `notes` DataFrame came from on Twitter's website. The file (the first file) is 9M rows. That crashed my little Google box a couple times. To be safe, I created a smaller file with just the last week of ratings.

In [ ]:
import pandas as pd

# remove the limits from the number of columns that can be shown
# and the number of characters in each cell
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# read in the notes and see what the columns are!
ratings = pd.read_csv("https://www.dropbox.com/scl/fi/hxt22duizfco4j9nuxcmm/ratings.csv?rlkey=9jlrfqrsyofz92sp2tvcphi9k&dl=1")
ratings.columns

In [ ]:
ratings.shape

That's just in one week!

Each row of this table is a rating of some note by a Community Notes participant. The [data dictionary](https://communitynotes.x.com/guide/en/under-the-hood/download-data) describes the columns. The `participantId` indicates who submitted each rating. These ID's appear also in the `notes` dataframe, as one person can both generate notes and rate the notes from others.

The `noteId` says which note was rated. This ID matches those in the `notes` table. This is the soul of the "relational database" model. Data are divided into different tables, the division providing a logical organization of the data. More on that at the end of the notebook.

The `helpfulnessLevel` holds the rater's answer as to whether they thought the note was `NOT_HELPFUL`, `SOMEWHAT_HELPFUL`, or `HELPFUL`.

There are other columns, but we'll focus on these initially. Let's have a look.

In [ ]:
ratings[["raterParticipantId","noteId","helpfulnessLevel"]]

Here we might look at the different counts of ratings per `noteId`. Some have quite a lot of activity.

In [ ]:
ratings.value_counts("noteId")

We can look up the most rated note in the `notes` table and see what was said...

In [ ]:
notes[notes["noteId"]==1760782212792807773]

... and to decide if this often-rated note was useful, we can look at it's `helpfulnessLevel`. A simple `.value_counts()` will do.

In [ ]:
ratings[ratings["noteId"]==1760782212792807773].value_counts("helpfulnessLevel")

Let's see how each note performed in terms of its ratings. We can use our new friend `.groupby()` to cluster the data by `noteId` and `helpfulnessLevel` and then count how many times each occurs.

In [ ]:
tmpratings = ratings.groupby(["noteId","helpfulnessLevel"],as_index=False)["raterParticipantId"].count()
tmpratings

**Table manipulations**

First, notice that our data sets, `tweets`, `notes`, and `ratings` all deal with different kinds of things. They link together through a system of IDs. Tweets are referenced in both the `notes` and `ratings` tables through the `tweetId`. And in the `tweets` table, there is a unique `tweetId` per tweet.

We have then linked across (manually), to find which tweet text is associated with a collection of notes, say. Similarly, we could collect all the ratings for a given note and then pull that note text, or the indications that the note's creator left in terms of its factual correctness.

This division of data into separate tables is the hallmark of what's often called the "relational data model". Quoting [here](https://www.tutorialspoint.com/Relational-Database-Model), the model consists of

>**Relations and attributes**: The various tables and attributes related to each table are identified. The tables represent entities, and the attributes represent the properties of the respective entities.
<br><br>
**Primary keys**: The attribute or set of attributes that help in uniquely identifying a record is identified and assigned as the primary key.
<br><br>
**Relationships**: The relationships between the various tables are established with the help of foreign keys. Foreign keys are attributes occurring in a table that are primary keys of another table. The types of relationships that can exist between the relations (tables) are One to one, One to many, and Many to many
<br><br>
**Normalization**: This is the process of optimizing the database structure. Normalization simplifies the database design to avoid redundancy and confusion.

In our case, the entities are tweets and notes and ratings. The primary keys are the `tweetId` for `tweets`, the `noteId` for `notes` and for `ratings` it's the pair `noteId` and `raterParticipantId` (making the pair a "composite primary key").

In [ ]:
ratings.value_counts(["noteId","raterParticipantId"])

The relational model has a lot going for it. By dividing the data into distinct entities, we have information in only one place. Suppose we need to correct a tweet or make a change in one of the notes. We have to touch just one table to make the change. (As opposed to including the original tweet text along with each note, say.)

But that does leave us wanting to perform analyses that joins different data sets. In pandas, we can "merge" data sets together to create new data relations. The function is, um, `merge()`. There are several ways to perform this operation. (Left and right here refer to the two dataframes in the `merge` call - one is on the left and one is on the right.)

* **left**: use only keys from left frame

* **right**: use only keys from right frame

* **outer**: use union of keys from both frames

* **inner**: use intersection of keys from both frames

* **cross**: creates the cartesian product from both frames

Let's look at a couple different examples, adding tweet data to the information about each note.

In [ ]:
notes.shape

In [ ]:
# add tweets to notes where you can, leave NA where you can't
m1 = pd.merge(notes,tweets,on="tweetId",how="left")
m1.shape

In [ ]:
# drop all notes that don't have tweets in our data set
m1 = pd.merge(notes,tweets,on="tweetId",how="inner")
m1.shape

Remember, we just have 1400 tweets and so not all the notes will have their tweet in the set.

Now, try it yourself!

In [ ]:
# your code here



Let's go back to our ratings breakdown by note.

In [ ]:
tmpratings = ratings.groupby(["noteId","helpfulnessLevel"],as_index=False)["raterParticipantId"].count()
tmpratings


Suppose next our goal is to `merge()` a summary of the rating data with our `notes` DataFrame. Then this DataFrame isn't quite what we want. This is a "long" form of the data, with up to three rows per note. Instead we'd prefer a "wide" data set.

That is, instead of stretchng out the ratings in the column `helpfulnessLevel`, we can create three columns, one for each of HELPFUL, NOT_HELPFUL and SOMEWHAT_HELPFUL. Our data for the first note might look like this

```
noteId	            HELPFUL	NOT_HELPFUL SOMEWHAT_HELPFUL
1354678831655735296	      4             1                0
```

See? Long to wide.

The tool we need to do this is an old fashioned pivot table. Here we group the entries in `tmpratings` by `noteId`, pull data from down the `helpfulnessLevel` column and use the counts in the `raterParticipantId` column to fill in columns for the three kinds of ratings.

We introduce 0's if some rating wasn't present for a note.

In [ ]:
ratingcounts = pd.pivot_table(tmpratings,index="noteId",columns="helpfulnessLevel",values="raterParticipantId",fill_value=0).reset_index()
ratingcounts

In pictures, this is what `pivot_table()` is doing. (The figure from the pandas web site refers to "stacked" and "records" versus "long" and "wide".

<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_pivot.png" width=500>

Finally, let's merge this data set into the `newnotes` data we created earlier -- this contained data about notes made this month.

Here we are merging on the left, meaning we are taking each row of `newnotes` and looking up the rating data in `ratingcounts` and adding it if we find it.

If we don't find it, we fill the rating data with 0's, as it means none of the Community Notes raters rated it yet.

In [ ]:
# recall newnotes was just the most recent week of notes
newnotes = notes[notes["createdAtDatetime"] > "2024-02-01"]

# merge newnotes with the rating counts to see how each performed.
# we use fillna(0) to turn all the missing values in the data to 0's.
# the missing values come from notes that have not been rated and so
# the NaN's are legitimately 0
newnotes = pd.merge(newnotes,ratingcounts,on="noteId",how="left").fillna(0)
newnotes.columns

In [ ]:
newnotes.shape

And now let's look... Here I keep just a few columns again.

In [ ]:
newnotes[["summary","date","HELPFUL","NOT_HELPFUL","SOMEWHAT_HELPFUL"]]

In [ ]:
newnotes = pd.merge(newnotes,tweets[["name","tweetId","tweet"]],on="tweetId",how="left")
newnotes.columns

In [ ]:
newnotes[["name","tweet","summary","date","HELPFUL","NOT_HELPFUL","SOMEWHAT_HELPFUL"]][~newnotes["tweet"].isna()]

I've gone through the details of forming these different DataFrames to show you the kind of prep work we might go through to start analyzing a reasonably rich data set.

**7. Your turn. We now have a lot of data. We have a dataframe of notes from this month called `newnotes.` It includes ratings for each note from the Birdwatch community. And we have a collection of tweets that these notes refer to called `tweets`. Come up with a question about these data and write the code to answer it. Try a few questions if you're feeling particularly interested in how Birdwatch works.**

Your question goes here

In [ ]:
# your code here



Consider where we might go next. Watching this platform evolve or implode over the next few weeks would be an amazing project for the class.

Before we leave, a comment on tables. Keep in mind the underlying data here. We don't have a single file, but we have three, each one examining a different aspect of the Birdwatch program. One holds notes, one holds ratings and one tracks the history of a note as it is considered helpful or not.

This how a "relational model" database model works. Tables are linked together by keys. Our ratings table, for example, holds data about each rating that Birdwatchers contributed. Each rating refers to a note (via the `noteId`) another participant left behind. The details of that note are in the notes table.

By putting all the data about ratings in one place, separate from that for the notes, we achieve some (at least logical) efficiency. As ratings come in, they do not change the note, its `summary` text or the time it was created or the tweet it refers to. This note-specific data is kept in one place.

We "merge" these tables together when we need a bigger story.
<br>
<br>
